# Shortest Route: Finding the most efficient driving routes between New York City landmarks

## Description of Approach

##### **Approach**
- We selected NYC because it's a group member's former home
- Nodes were defined by selection iconic NYC locations (ex. Times Square, Rockefeller Center, Empire State Building, etc.)
- The map was built using .......
- For both base and rush hour scenarios, we used Google Maps data that included travel times in normal and peak traffic conditions

##### **Assumptions**
- The time from node A to node J is the same as the time from node J to node A

## Graph Structure & Algorithm Implementation

We chose to represent our graph with ........

Dijkstra's Algorithm was implemented by ........

Design choices include ..........

## Code-Based Verification

### Import Libraries, Functions, Data

In [1]:
import sys
import os
import json
import folium

project_root = os.path.abspath(os.path.join('..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from modules.graph import Graph
from modules.mapGraph import build_graph_from_json
from modules.displayMap import build_graph_mappings

In [3]:
with open('manhattan_driving_distances.json', 'r') as f:
    loaded_dict = json.load(f)

### Base Case: Average Traffic Conditions

In [4]:
# average traffic time
start_node = "Times Square"

g = Graph(len(loaded_dict['nodes']))

for key in loaded_dict['nodes'].keys():
    g.add_vertex_data(int(key), loaded_dict['nodes'][key]["name"])

for i in loaded_dict["pairwise"].keys():
    for j in loaded_dict["pairwise"][i].keys():
        g.add_edge(int(i), int(j), loaded_dict["pairwise"][i][j]['avg_time_min'])

distances = g.dijkstra(start_node)

for i, d in enumerate(distances):
    print(f"Average Travel Time from \033[1m{start_node}\033[0m to \033[1m{g.vertex_data[i]}\033[0m: {round(d,2)} minutes")

Average Travel Time from Times Square to Times Square: 0 minutes
Average Travel Time from Times Square to Rockefeller Center: 2.45 minutes
Average Travel Time from Times Square to Bryant Park / NYPL: 1.76 minutes
Average Travel Time from Times Square to Grand Central Terminal: 4.33 minutes
Average Travel Time from Times Square to Empire State Building: 4.31 minutes
Average Travel Time from Times Square to Flatiron / Madison Sq Park: 8.17 minutes
Average Travel Time from Times Square to Union Square: 10.26 minutes
Average Travel Time from Times Square to Chelsea Market: 15.4 minutes
Average Travel Time from Times Square to Washington Square Park: 14.18 minutes
Average Travel Time from Times Square to Battery Park / South Ferry: 29.16 minutes


#### Spotlight: Times Square & Washington Square Park

In [5]:
print(f"Average Travel Time from \033[1m{start_node}\033[0m to \033[1mWashington Square Park\033[0m: {round(distances[8],2)} min")
print(f"Average Travel Time from \033[1mWashington Square Park\033[0m to \033[1m{start_node}\033[0m: {round(distances[8],2)} min")

Average Travel Time from Times Square to Washington Square Park: 14.18 min
Average Travel Time from Washington Square Park to Times Square: 14.18 min


In [6]:
build_graph_mappings(loaded_dict, "Times Square", "Washington Square Park", "avg")

Shortest path indices: [0, 2, 4, 5, 6, 8]
Shortest path names: ['Times Square', 'Bryant Park / NYPL', 'Empire State Building', 'Flatiron / Madison Sq Park', 'Union Square', 'Washington Square Park']


### Rush Hour Scenario: Peak Traffic Conditions

In [7]:
# peak traffic time
start_node = "Times Square"

g = Graph(len(loaded_dict['nodes']))

for key in loaded_dict['nodes'].keys():
    g.add_vertex_data(int(key), loaded_dict['nodes'][key]["name"])

for i in loaded_dict["pairwise"].keys():
    for j in loaded_dict["pairwise"][i].keys():
        g.add_edge(int(i), int(j), loaded_dict["pairwise"][i][j]['peak_time_min'])

distances = g.dijkstra(start_node)

for i, d in enumerate(distances):
    print(f"\033[1mPeak\033[0m Travel Time from \033[1m{start_node}\033[0m to \033[1m{g.vertex_data[i]}\033[0m: {round(d,2)} minutes")

Peak Travel Time from Times Square to Times Square: 0 minutes
Peak Travel Time from Times Square to Rockefeller Center: 4.9 minutes
Peak Travel Time from Times Square to Bryant Park / NYPL: 3.52 minutes
Peak Travel Time from Times Square to Grand Central Terminal: 8.66 minutes
Peak Travel Time from Times Square to Empire State Building: 8.62 minutes
Peak Travel Time from Times Square to Flatiron / Madison Sq Park: 16.34 minutes
Peak Travel Time from Times Square to Union Square: 20.52 minutes
Peak Travel Time from Times Square to Chelsea Market: 30.8 minutes
Peak Travel Time from Times Square to Washington Square Park: 28.36 minutes
Peak Travel Time from Times Square to Battery Park / South Ferry: 58.32 minutes


#### Spotlight: Times Square & Washington Square Park

In [8]:
print(f"Average Travel Time from \033[1m{start_node}\033[0m to \033[1mWashington Square Park\033[0m: {round(distances[8],2)} min")
print(f"Average Travel Time from \033[1mWashington Square Park\033[0m to \033[1m{start_node}\033[0m: {round(distances[8],2)} min")

Average Travel Time from Times Square to Washington Square Park: 28.36 min
Average Travel Time from Washington Square Park to Times Square: 28.36 min


In [9]:
build_graph_mappings(loaded_dict, "Times Square", "Washington Square Park", "peak")

Shortest path indices: [0, 2, 4, 5, 6, 8]
Shortest path names: ['Times Square', 'Bryant Park / NYPL', 'Empire State Building', 'Flatiron / Madison Sq Park', 'Union Square', 'Washington Square Park']


## Discussion & Analysis

- How routes changed
- Impact of uncertainty
- Interpretation of results
- Assumptions & limitations